# National Coral Reef Monitoring Project shapefile to geojson

This notebook attempts to accomplish the following activities:

 - [ ] Convert shapefiles to geojson
 - [ ] Create a combined geojson from multiple shapefiles
 - [ ] Use different plotting techniques to look at the data

## First lets make some functions to make our lives easier

In [ ]:
import geopandas as gpd
import pandas as pd

def shape2geojson(infile: str):
    try:
        infile.split(".")[-1] != 'shp'

    except:
        return ("Incorrect input file!\n%s is not a shapefile." % infile)

    else:
        try:
            open(infile, 'r')

        except FileNotFoundError:
            return ("File %s not found!" % infile)

        else:
            shpfile = gpd.read_file( infile ).to_crs( "EPSG:4326" )
            outfile = infile.replace( '.shp', '.geojson' )
            shpfile.to_file( outfile, driver='GeoJSON' )

            return True
        
        
def multishape2geojson(shapefiles: list, outfile: str):

    gdf = pd.concat([
                    gpd.read_file(shp).to_crs("EPSG:4326")
                    for shp in shapefiles
                    ]).pipe(gpd.GeoDataFrame)

    gdf['geometry'].to_file(outfile, driver='GeoJSON')


## convert shapefiles to geoJSON

In [ ]:
# import glob

# shapefiles = glob.glob('./NCRMP/**/*.shp', recursive=True)

# A bunch of individual files
# for shapefile in shapefiles:
#     shape2geojson(shapefile)

# One large file
# multishape2geojson(shapefiles, 'NCRMP_combined_geometry.geojson')

## Read geojson and make simple plot

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('../NCRMP/NCRMP_combined_geometry.geojson')
gdf.plot( figsize=(200, 200) )

## Add geoJSON files to interactive map

In [ ]:
import geopandas as gps
import pandas as pd
import glob

gjsons = glob.glob('../NCRMP/**/*.geojson', recursive=True)
gjsons

In [ ]:
import folium

m = folium.Map(
    location=[0, 0],
    #tiles="cartodbpositron",
    zoom_start=2,
)

for gjson in gjsons:
    if "NCRMP_combined" not in gjson:
        
        name = gjson.split("\\")[-1].split(".")[0]
        
        folium.GeoJson(gjson, name=name ).add_to(m)
        
folium.LayerControl().add_to(m)

m